In [7]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('ml').getOrCreate()
spark

In [8]:
train=spark.read.csv('ps6.csv',header=True,inferSchema=True)
train.show()

+------+---+----------+------+
|  Name|Age|Experience|Salary|
+------+---+----------+------+
|  Fred| 31|        10| 30000|
|  Jack| 30|         8| 25000|
|Arevan| 29|         4| 20000|
|  Cara| 24|         3| 20000|
| Ethan| 21|         1| 15000|
|  Mary| 23|         2| 18000|
+------+---+----------+------+



In [9]:
train.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [10]:
train.columns

['Name', 'Age', 'Experience', 'Salary']

In [11]:
# [Age,Experience]  # new feature ----->independent feature group

In [17]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['Age','Experience'],outputCol='Independent Features')

In [18]:
output=featureassembler.transform(train)
output.show()

+------+---+----------+------+--------------------+
|  Name|Age|Experience|Salary|Independent Features|
+------+---+----------+------+--------------------+
|  Fred| 31|        10| 30000|         [31.0,10.0]|
|  Jack| 30|         8| 25000|          [30.0,8.0]|
|Arevan| 29|         4| 20000|          [29.0,4.0]|
|  Cara| 24|         3| 20000|          [24.0,3.0]|
| Ethan| 21|         1| 15000|          [21.0,1.0]|
|  Mary| 23|         2| 18000|          [23.0,2.0]|
+------+---+----------+------+--------------------+



In [19]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [20]:
finalized_data=output.select('Independent Features','Salary')
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [21]:
from pyspark.ml.regression import LinearRegression

In [40]:
#train -test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25]) #split 75% and 25%

In [41]:
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [42]:
regressor.coefficients

DenseVector([-258.8832, 1670.0508])

In [43]:
regressor.intercept

20543.147208121274

In [45]:
#prediction

pred_results=regressor.evaluate(test_data)

In [46]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16776.649746192812|
|          [24.0,3.0]| 20000|19340.101522842593|
+--------------------+------+------------------+



In [47]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1218.2741116751095, 1795975.1604008244)

In [ ]:
#check